### Meta Class2

- Type함수는 3개의 인자를 받는다 → name, base, dct

In [7]:
# type 동적 클래스 생성 예제

# 정적 클래스 생성

class Sample1():
    pass

# 동적 클래스 생성

s1 = type('Sample1',(),{}) # 이거 자체로 class로 만든 것

print('Ex1 > ',s1)
print('Ex1 > ',type(s1))
print('Ex1 > ',s1.__base__)
print('Ex1 > ',s1.__dict__)

# 동적 생성 + 상속

class Parent1:
    pass

class Sample2(Parent1):
    attr1=100
    attr2='hi'

s2=type(
    'Sample2',
    (Parent1,),
    dict(attr1=100, attr2='hi')
) 
# Parent1을 상속받는다는 것 / {'attr1':100, 'attr2':'hi'}로 dict 표현할 수도 있음

print('Ex2 > ',s2)
print('Ex2 > ',type(s2))
print('Ex2 > ',s2.__base__)
print('Ex2 > ',s2.__dict__)
print('Ex2 > ',s2.attr1, s2.attr2)

Ex1 >  <class '__main__.Sample1'>
Ex1 >  <class 'type'>
Ex1 >  <class 'object'>
Ex1 >  {'__module__': '__main__', '__dict__': <attribute '__dict__' of 'Sample1' objects>, '__weakref__': <attribute '__weakref__' of 'Sample1' objects>, '__doc__': None}
Ex2 >  <class '__main__.Sample2'>
Ex2 >  <class 'type'>
Ex2 >  <class '__main__.Parent1'>
Ex2 >  {'attr1': 100, 'attr2': 'hi', '__module__': '__main__', '__doc__': None}
Ex2 >  100 hi


In [14]:
# attribute뿐만 아니라 메소드도 동적으로 만들어보기

class SampleEx:
    attr1=30
    attr2=100
    
    def add(self,m,n):
        return m+n
    
    def mul(self,m,n):
        return m*n
    
ex = SampleEx()

print('Ex3 > ', ex.attr1)
print('Ex3 > ', ex.attr2)
print('Ex3 > ', ex.add(100,200))
print('Ex3 > ', ex.mul(100,20))

s3 = type('Sample3',(object,),dict(attr1=30, attr2=100, add =lambda x,y:x+y, mul=lambda x,y:x*y)) # 모든 클래스는 object를 상속받기에
#{'attr1':30, 'attr2':100, 'add' : lambda x,y:x+y, 'mul':lambda x,y:x*y}



print('Ex4 > ', s3.attr1)
print('Ex4 > ', s3.attr2)
print('Ex4 > ', s3.add(100,200))
print('Ex4 > ', s3.mul(100,20))

Ex3 >  30
Ex3 >  100
Ex3 >  300
Ex3 >  2000
Ex4 >  30
Ex4 >  100
Ex4 >  300
Ex4 >  2000


### 메타 클래스 상속
1. type 클래스 상속
2. metaclass 속성 사용
3. 커스텀 메타 클래스 생성

In [21]:
# 커스텀 메타클래스 생성 예제(Type 상속 X)

def cus_mul(self,d): #self는 instance 자기 자신
    for i in range(len(self)): # 리스트, 튜플 등 반복가능한 자료형을 받아서 len길이만큼 돌린다.
        self[i] = self[i]*d # 함수 호출할 때, self랑 d 두 개 넘어와야 함
        
def cus_replace(self,old,new):
    while old in self: #while문에서 사용되었기에 self는 시퀀스형 데이터라는 것
        self[self.index(old)] = new
        
# list 상속받음, 메소드 2개 추가
CustomList1 = type(
    'CustomList1',
    (list,),
    {
        'desc':'커스텀 리스크1', #name 
        'cus_mul':cus_mul,  #bases
        'cus_replace':cus_replace #namespace
    }
)
        
c1 = CustomList1([1,2,3,4,5,6,7,8,9])
c1.cus_mul(1000)
print('Ex1 > ', c1)
c1.cus_replace(1000,7777)
print('Ex1 > ', c1)
print('Ex1 > ', c1.desc)
print('Ex1 > ', dir(c1))

# list.index
# a = [123, 421, 212, 11, 24, 102, 29, 92, 10]
# a.index(421) → 1


Ex1 >  [1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000]
Ex1 >  [7777, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000]
Ex1 >  커스텀 리스크1
Ex1 >  ['__add__', '__class__', '__class_getitem__', '__contains__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__iadd__', '__imul__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__rmul__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'append', 'clear', 'copy', 'count', 'cus_mul', 'cus_replace', 'desc', 'extend', 'index', 'insert', 'pop', 'remove', 'reverse', 'sort']


In [26]:
# Ex2

# 커스텀 메타클래스 생성 예제(Type 상속 O)

# class MetaClassName(type):
#    def __new__(metacls, name, bases, namespace)
#    코드

# 실행순서: new -> init -> call 순서

class CustomListMeta(type):    
    # 생성된 인스턴스 초기화
    def __init__(self, object_or_name, bases, dict):
        print('__init__ --> ', self, object_or_name, bases, dict)
        dict # 후킹
        super().__init__(object_or_name, bases, dict)
        
    # 인스턴스 실행
    def __call__(self, *args, **kwargs):
        print('__call__ --> ', self, *args, **kwargs)
        return super().__call__(*args, **kwargs)
    
    # 클래스 인스턴스 생성(메모리 초기화)
    def __new__(metacls, name, bases, namespace):
        print('__new__ -->',metacls, name, bases, namespace)
        namespace['desc'] = '커스텀리스트2'
        namespace['cus_mul'] = cus_mul
        namespace['cus_replace'] = cus_replace
        
        return type.__new__(metacls,name,bases,namespace)
    
CustomList2 = CustomListMeta('CustomList2',(list,),{})
c2 = CustomList2([1,2,3,4,5,6,7,8,9])
c2.cus_mul(1000)
print('Ex2 > ', c2)
c2.cus_replace(1000,7777)
print('Ex2 > ', c2)
print('Ex2 > ', c2.desc)

__new__ --> <class '__main__.CustomListMeta'> CustomList2 (<class 'list'>,) {}
__init__ -->  <class '__main__.CustomList2'> CustomList2 (<class 'list'>,) {'desc': '커스텀리스트2', 'cus_mul': <function cus_mul at 0x0000021DFAEAD3A0>, 'cus_replace': <function cus_replace at 0x0000021DFAEAD260>}
__call__ -->  <class '__main__.CustomList2'> [1, 2, 3, 4, 5, 6, 7, 8, 9]
Ex2 >  [1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000]
Ex2 >  [7777, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000]
Ex2 >  커스텀리스트2
